In [2]:
import json
import pandas as pd
from typing import Dict, List

In [3]:
with open(
    "../../../public_test_and_submission_guidelin/public_test_and_submission_guidelines/file_name_to_image_id.json",
    "r",
) as file:
    filename_2_id = json.load(file)

In [4]:
backends = [
    "opencv",
    "ssd",
    "dlib",
    "mtcnn",
    "retinaface",
    "mediapipe",
    "yolov8",
    "yunet",
    "fastmtcnn",
]
filenames = [be + "-output.csv" for be in backends]
data = {}
for backend, filename in zip(backends, filenames):
    try:
        data[backend] = pd.read_csv(filename)
    except Exception as e:
        print(e)

[Errno 2] No such file or directory: 'dlib-output.csv'
[Errno 2] No such file or directory: 'yolov8-output.csv'


In [5]:
def print_unique(data: Dict[str, pd.DataFrame]):
    all_data = pd.concat(data.values())
    print(all_data.age.unique())
    print(all_data.dominant_gender.unique())
    print(all_data.dominant_race.unique())
    print(all_data.dominant_emotion.unique())

In [6]:
print_unique(data)

[43 35 50 34 33 25 46 24 28 37 41 27 21 26 30 39 31 29 18 36 20 40 32 23
 38 19 48 22 49 42 45 57 17 44 51 53 52 47 55 15 59 58 16 66 14 54 56 11
 60 61 64 13 62 65 67 63]
['Woman' 'Man']
['asian' 'white' 'latino hispanic' 'black' 'middle eastern' 'indian']
['neutral' 'fear' 'sad' 'happy' 'angry' 'disgust' 'surprise']


In [7]:
def convert_age(age) -> str:
    if age <= 4:
        return "Baby"
    elif age <= 12:
        return "Kid"
    elif age <= 19:
        return "Teenager"
    elif age <= 39:
        return "20-30s"
    elif age <= 59:
        return "40-50s"
    else:
        return "Senior"


def convert_gender(gender: str) -> str:
    if gender == "Man":
        return "Male"
    if gender == "Woman":
        return "Female"


def convert_emotion(emo: str) -> str:
    if emo == "angry":
        return "Anger"
    elif emo == "sad":
        return "Sadness"
    elif emo == "happy":
        return "Happiness"
    elif emo == "neutral":
        return "Neutral"
    elif emo == "fear":
        return "Fear"
    elif emo == "disgust":
        return "Disgust"
    elif emo == "surprise":
        return "Surprise"


def convert_race(race: str) -> str:
    if race == "black":
        return "Negroid"
    elif race == "white" or "hispanic" in race:
        return "Caucasian"
    else:
        return "Mongoloid"


import ast


def convert_region(region_literal: str) -> str:
    region = ast.literal_eval(region_literal)
    return ast.literal_eval(repr([region["x"], region["y"], region["w"], region["h"]]))


def get_id(filename: str) -> str:
    return filename_2_id[filename]

In [8]:
out_dats = {}
for name, dat in data.items():
    out_dat = dat.copy()
    out_dat = out_dat.drop(columns=["race", "gender", "emotion", "face_confidence"]).rename(
        columns={"dominant_race": "race", "dominant_gender": "gender", "dominant_emotion": "emotion", "region": "bbox"}
    )
    out_dat.age = out_dat.age.apply(convert_age)
    out_dat.race = out_dat.race.apply(convert_race)
    out_dat.emotion = out_dat.emotion.apply(convert_emotion)
    out_dat.gender = out_dat.gender.apply(convert_gender)
    out_dat.bbox = out_dat.bbox.apply(convert_region)
    out_dat["image_id"] = out_dat.file_name.apply(get_id)
    out_dat = out_dat.reindex(
        columns=["file_name", "bbox", "image_id", "race", "age", "emotion", "gender"]
    ).sort_values("image_id")
    out_dats[name] = out_dat

In [9]:
out_dats

{'opencv':            file_name                   bbox  image_id       race     age  \
 1562   100147591.jpg  [1070, 488, 267, 267]         1  Caucasian  20-30s   
 1805   100868012.jpg  [1020, 270, 119, 119]         4  Mongoloid  20-30s   
 333    101013830.jpg     [250, 261, 70, 70]         5  Caucasian  20-30s   
 1584   101564605.jpg   [640, 252, 316, 316]         8  Mongoloid  40-50s   
 395    101954623.jpg  [1220, 357, 214, 214]         9  Mongoloid  20-30s   
 ...              ...                    ...       ...        ...     ...   
 1034  image_9841.jpg   [147, 192, 728, 728]      2180  Caucasian  20-30s   
 402   image_9913.jpg   [173, 225, 660, 660]      2181  Caucasian  20-30s   
 990   image_9915.jpg   [147, 183, 731, 731]      2182  Caucasian  20-30s   
 1134  image_9935.jpg   [142, 183, 732, 732]      2183  Caucasian  20-30s   
 258   image_9975.jpg   [183, 246, 641, 641]      2184  Mongoloid  20-30s   
 
         emotion  gender  
 1562    Sadness    Male  
 1805    S

In [10]:
import os
import zipfile

if not os.path.exists("submission/"):
    os.mkdir("submission/")

for name, dat in out_dats.items():
    dat.to_csv("answer.csv", index=False)

    with zipfile.ZipFile(f"submission/{name}-answer.zip", "w") as zipf:
        zipf.write("answer.csv")